# Hollow Sphere Mesh Generator (Gmsh)

This notebook creates a 3D tetrahedral mesh of a *hollow sphere* using **Gmsh**.

### Inputs
- `r_out`: outer radius
- `r_in`: inner radius (must be `< r_out`)
- `lc`: target mesh size (characteristic length)
- `p`: element order (1 for linear, 2 for quadratic, ...)

It writes a `.geo` file and (optionally) calls Gmsh to generate a `.msh` file if Gmsh is available.

In [ ]:

# ---- Parameters ----
r_out = 1.0   # outer radius
r_in  = 0.9   # inner radius
lc    = 0.02  # target mesh size
p     = 1    # element order (1=linear, 2=quadratic, ...)

geo_path = "hollow_sphere.geo"        # output .geo file
msh_path = f"hollow_sphere_rin{r_in:.1e}_rout{r_out:.1e}_lc{lc:.1e}_p{p}.msh"        # output .msh file (if gmsh is available)

assert r_in > 0, "r_in must be positive."
assert r_out > 0, "r_out must be positive."
assert r_out > r_in, "r_out must be greater than r_in."
assert lc > 0, "lc must be positive."
assert p in (1, 2, 3), "p should be 1, 2 or 3 (common choices)."


In [2]:

# ---- Generate the .geo file content for a hollow sphere ----
# We use OpenCASCADE to create two spheres and take their boolean difference.
# We also tag physical groups: the 3D volume and the inner & outer spherical surfaces (via bounding boxes).

geo_file = f"""SetFactory("OpenCASCADE");

// Parameters
r_out = {r_out};
r_in  = {r_in};
lc    = {lc};
p     = {p};

// Build solids
Sphere(1) = {{0, 0, 0, r_out}};
Sphere(2) = {{0, 0, 0, r_in}};

// Hollow shell = outer sphere minus inner sphere
out() = BooleanDifference{{ Volume{{1}}; Delete; }}{{ Volume{{2}}; }};

// Mesh sizing
Mesh.CharacteristicLengthMin = lc;
Mesh.CharacteristicLengthMax = lc;
Mesh.ElementOrder = p;

// Tag the volume
Physical Volume("hollow_sphere") = {{ out() }};

// Tag inner & outer spherical surfaces using bounding boxes around r_in and r_out.
// We add a small epsilon to make selection robust.
eps = r_out * 1e-6;

// All boundary surfaces of the hollow volume
all_surf() = Boundary{{ Volume{{ out() }}; }};

// Inner surfaces: near radius r_in
inner() = Surface In Bounding Box{{ -r_in - eps, -r_in - eps, -r_in - eps,  r_in + eps,  r_in + eps,  r_in + eps }};

// Outer surfaces: near radius r_out (subtract inner from all if both selected)
outer_guess() = Surface In Bounding Box{{ -r_out - eps, -r_out - eps, -r_out - eps,  r_out + eps,  r_out + eps,  r_out + eps }};

// Sometimes Bounding Box picks both; intersect with all_surf to stay on the shell.
outer() = CombinedBoundary{{ all_surf() }};
Physical Surface("inner") = {{ inner() }};
Physical Surface("outer") = {{ outer_guess() }};

// 3D meshing algorithm (optional; use defaults if unsure)
Mesh.Algorithm3D = 1; // Delaunay
"""

# Write the .geo file
with open(geo_path, "w") as f:
    f.write(geo_file)

print(f"Wrote {geo_path}")
print("Preview of the first ~60 lines:")
print("\n".join(geo_file.splitlines()[:60]))


Wrote hollow_sphere.geo
Preview of the first ~60 lines:
SetFactory("OpenCASCADE");

// Parameters
r_out = 1.0;
r_in  = 0.9;
lc    = 0.05;
p     = 2;

// Build solids
Sphere(1) = {0, 0, 0, r_out};
Sphere(2) = {0, 0, 0, r_in};

// Hollow shell = outer sphere minus inner sphere
out() = BooleanDifference{ Volume{1}; Delete; }{ Volume{2}; };

// Mesh sizing
Mesh.CharacteristicLengthMin = lc;
Mesh.CharacteristicLengthMax = lc;
Mesh.ElementOrder = p;

// Tag the volume
Physical Volume("hollow_sphere") = { out() };

// Tag inner & outer spherical surfaces using bounding boxes around r_in and r_out.
// We add a small epsilon to make selection robust.
eps = r_out * 1e-6;

// All boundary surfaces of the hollow volume
all_surf() = Boundary{ Volume{ out() }; };

// Inner surfaces: near radius r_in
inner() = Surface In Bounding Box{ -r_in - eps, -r_in - eps, -r_in - eps,  r_in + eps,  r_in + eps,  r_in + eps };

// Outer surfaces: near radius r_out (subtract inner from all if both selected)
outer_g

In [3]:

# ---- (Optional) Generate .msh with Gmsh, if available ----
import shutil, subprocess, sys

gmsh = shutil.which("gmsh")
if gmsh is None:
    print("Gmsh not found on PATH. Skipping mesh generation. You can run:")
    print(f"  gmsh -3 {geo_path} -o {msh_path}")
else:
    print(f"Using gmsh at: {gmsh}")
    cmd = [gmsh, "-3", geo_path, "-o", msh_path]
    print("Running:", " ".join(cmd))
    try:
        subprocess.run(cmd, check=True)
        print(f"Wrote {msh_path}")
    except subprocess.CalledProcessError as e:
        print("Gmsh failed with return code:", e.returncode)
        print("You can try running the command manually for more logs.")


Using gmsh at: /usr/bin/gmsh
Running: /usr/bin/gmsh -3 hollow_sphere.geo -o hollow_sphere_rin9.0e-01_rout1.0e+00_lc5.0e-02_p2.msh
Info    : Running '/usr/bin/gmsh -3 hollow_sphere.geo -o hollow_sphere_rin9.0e-01_rout1.0e+00_lc5.0e-02_p2.msh' [Gmsh 4.8.4, 1 node, max. 1 thread]
Info    : Started on Thu Aug 14 17:35:56 2025
Info    : Reading 'hollow_sphere.geo'...
Info    : Done reading 'hollow_sphere.geo'
Info    : Meshing 1D...
Info    : [ 20%] Meshing curve 5 (Circle)
Info    : [ 70%] Meshing curve 8 (Circle)
Info    : Done meshing 1D (Wall 0.000154565s, CPU 0.000156s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Sphere, Frontal-Delaunay)


Error   : 'hollow_sphere.geo', line 32: syntax error (Bounding)
Error   : 'hollow_sphere.geo', line 35: syntax error (Bounding)
Error   : 'hollow_sphere.geo', line 38: syntax error (()
Error   : 'hollow_sphere.geo', line 39: Unknown variable 'inner'
Error   : 'hollow_sphere.geo', line 40: Unknown variable 'outer_guess'


Info    : [ 50%] Meshing surface 3 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.604172s, CPU 0.604126s)
Info    : Meshing 3D...
Info    : 3D Meshing 2 volumes with 1 connected component
Info    : Tetrahedrizing 11060 nodes...
Info    : Done tetrahedrizing 11068 nodes (Wall 0.121045s, CPU 0.117024s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.266079s, CPU 0.262012s)
Info    : Found volume 2
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 19.3685 (nodes removed 0 0)
Info    : It. 500 - 500 nodes created - worst tet radius 2.8994 (nodes removed 0 0)
Info    : It. 1000 - 1000 nodes created - worst tet radius 2.35938 (nodes removed 0 0)
Info    : It. 1500 - 1500 nodes created - worst tet radius 2.08004 (nodes removed 0 0)
Info    : It. 2000 - 2000 nodes created - worst tet radius 1.90021 (nodes removed 0 0)


In [4]:

# ---- Quick check: show a few lines of the .msh file (if created) ----
from pathlib import Path
p_msh = Path(msh_path)
if p_msh.exists():
    with open(p_msh, "r", errors="ignore") as f:
        for i, line in enumerate(f):
            print(line.rstrip())
            if i > 100:
                break
else:
    print("No .msh file to preview.")


$MeshFormat
4.1 0 8
$EndMeshFormat
$PhysicalNames
3
2 2 "inner"
2 3 "outer"
3 1 "hollow_sphere"
$EndPhysicalNames
$Entities
4 6 2 2
3 5.51091059616309e-17 -1.349783804395672e-32 0.9 0
4 5.51091059616309e-17 -1.349783804395672e-32 -0.9 0
5 6.123233995736766e-17 -1.499759782661858e-32 1 0
6 6.123233995736766e-17 -1.499759782661858e-32 -1 0
4 -1.000000000554834e-07 -1.00000000055421e-07 0.8999999000000001 1.000000000554834e-07 1.00000000055421e-07 0.9000001 0 2 3 -3
5 -1.00000000169409e-07 -1.000000002204364e-07 -0.9000001 0.9000001 1e-07 0.9000001 0 2 4 -3
6 -1.000000000554834e-07 -1.00000000055421e-07 -0.9000001 1.000000000554834e-07 1.00000000055421e-07 -0.8999999000000001 0 2 4 -4
7 -1.000000000616482e-07 -1.000000000615789e-07 0.9999999000000001 1.000000000616482e-07 1.000000000615789e-07 1.0000001 0 2 5 -5
8 -1.000000002249202e-07 -1.000000002449294e-07 -1.0000001 1.0000001 1e-07 1.0000001 0 2 6 -5
9 -1.000000000616482e-07 -1.000000000615789e-07 -1.0000001 1.000000000616482e-07 1.00